In [3]:
import zlib
import hashlib
import os
from binascii import hexlify, unhexlify

In [4]:
def sha1_to_path(sha1, directory='.git'):
    prefix = sha1[:2]
    suffix = sha1[2:]
    path = '/'.join([directory, 'objects', prefix, suffix])
    return path
def sha1_to_directory(sha1, directory='.git'):
    prefix = sha1[:2]
    path = '/'.join([directory, 'objects', prefix])
    return path

In [5]:
def pretty_print(sha1, directory='.git'):
    path = sha1_to_path(sha1, directory)
    with open(path, mode='rb') as compressed:
        decompressed = zlib.decompress(compressed.read())
    print(decompressed)
    entries = decompressed.split(b'\x00')
    object_type = entries[0]
    objects = []
    for i,obj in enumerate(entries[1:]):
        objects.append(hexlify(obj) if i%2!=0 else obj)
    return [object_type, *objects]

In [6]:
pretty_print('8accee2c256b491ea0d9c138d0bbc94d064ba590')

b'tree 37\x00100644 README.md\x00\xad\x0e\xc6\xbc\x18\x16\x0cz\xa6\x05\x97Q\x0e\x90h\xa9\xdb\x8e\x02\xb8'


[b'tree 37', b'100644 README.md', b'ad0ec6bc18160c7aa60597510e9068a9db8e02b8']

In [7]:
def store_blob(content, directory='.git'):
    header = 'blob ' + str(len(content)) + '\x00'
    store = header + content
    hash = hashlib.sha1()
    hash.update(store.encode('utf8'))
    sha1 = hash.hexdigest()
    path = sha1_to_path(sha1, directory)
    if os.path.isfile(path):
        # Given how unlikely it is that there has been a hash collision,
        # assume the user is writing the same blob again.
        pass
    else:
        os.makedirs(sha1_to_directory(sha1, directory), exist_ok=True)
        with open(path, mode='wb') as file:
            file.write(zlib.compress(store.encode()))
    return sha1

In [8]:
store_blob('hello world')

'95d09f2b10159347eece71399a7e2e907ea3df4f'

In [9]:
store_blob('hello world')

'95d09f2b10159347eece71399a7e2e907ea3df4f'